In [ ]:
# Imports
!pip install --upgrade cpmpy --quiet
!pip install matplotlib --quiet
import cpmpy as cp
import numpy as np
import matplotlib.pyplot as plt

**Useful Resources:**
* CPMpy summary sheet: https://cpmpy.readthedocs.io/en/latest/summary.html
* CPMpy modeling documentation: https://cpmpy.readthedocs.io/en/latest/modeling.html


# Session 5: symmetry and dominance breaking

## Exercise 1: grocery store prices
A kid goes into a grocery store and buys four items. The cashier charges €7.11,
the kid pays and is about to leave when the cashier calls the kid back, and says
"Hold on, I multiplied the four items instead of adding them; I'll try again; Hah,
with adding them the price still comes to €7.11". What were the prices of the
four items?
Write a model that solves the problem. Use solveAll to find all the solutions

Hint: CPMpy does not work well with floating point numbers, think about how to convert them to integers

In [ ]:
#example code, change this
a,b = cp.intvar(1,5, shape=2, name=list("ab"))
model = cp.Model(a + b > 5)
model.solveAll(display=[a,b])

Now think about the possible symmetric solutions and add the corresponding constraints to exclude them from the satisfying assignments.
Find all solutions to check that there are no symmetric solutions among the possible ones.
What kind of symmetries are present in this problem, is it a problem, instance or model symmetry?
Value or variables symmetry?

In [ ]:
# add symmetry breaking constraints here

## Exercise 2
We want to climb a stair of n steps: we can move up one step at a time or take
two steps at a time (you can generalize to m steps at a time). We are interested
in the possible combinations of steps taken each time to get to the top of the staircase.
For example, a stair of 4 steps with m=2 can be climbed with a sequence of
four one-step moves or with two two-step moves. Define a model that
finds all the possible combinations of moves of size 1, 2 up to m to climb a staircase
of size n.
For n=4 and m=2 the solutions without symmetries for example are:
1111
112
121
211
22


Think about all the possible symmetric solutions
Is it a problem, instance or model symmetry? Value or variables symmetry?
Add constraints that break these symmetries.
Make sure to set the solver to find all solutions in order to verify that your model does not lead to symmetric solutions.

In [ ]:
# add symmetry breaking


Assume now that we do not care for the order of the sequence of steps: Thus 112 and 121 would be symmetric solutions. Can we break these symmetries by reformulation?


## Exercise 3
We revisit the RCPSP problem from exercise session 3, with a few changes.
A number of activities are to be scheduled. Each activity has a duration and cannot be interrupted. There are no precedence relations between activities. There are 2 renewable resources, each with a maximum capacity of 1, ensuring no more than 1 unit of each resource can be in use at any given time slot. Each activity has a demand on each resource.
Minimize the starting time of the last activity.
What (types of) symmetries do you see?
Is there any dominance breaking possible?


In [ ]:
## Data, there are 12 tasks to be scheduled, with 2 resources that can be used.
# Task durations
durations = cp.cpm_array([3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5])
# resources needed by the tasks
resource_needs = cp.cpm_array([[1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0]])
# Total resource capacity
resource_capacities = cp.cpm_array([1, 1])
nb_resource = len(resource_capacities)
nb_jobs = len(durations)
max_duration = sum(durations)  # dummy upper bound, can be improved of course

start_time = cp.intvar(0, max_duration, shape=nb_jobs)

model = cp.Model()

In [ ]:
# Visualize the RCPSP solution
model.solve()

# Get the solution values
start_times = start_time.value()
print(start_times)
end_times = start_times + np.array(durations)

# Create a figure and axis
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), gridspec_kw={'height_ratios': [3, 1]})

# Define colors for each resource
colors = ['#FF9999', '#66B2FF']

# Plot each task as a horizontal bar
for i in range(nb_jobs):
    if durations[i] > 0:  # Skip tasks with zero duration
        ax1.barh(i, durations[i], left=start_times[i], height=0.5,
                 color=colors[np.argmax(resource_needs[i])],
                 alpha=0.8, edgecolor='black')

        # Add task labels
        ax1.text(start_times[i], i, f'Task {i}', va='center', ha='right', fontweight='bold')

# Customize the task plot
ax1.set_xlabel('Time')
ax1.set_ylabel('Tasks')
ax1.set_title('RCPSP Solution Visualization')
ax1.set_ylim(-1, nb_jobs)
ax1.invert_yaxis()  # Invert y-axis to have task 0 at the top

# Add a legend for tasks
legend_elements = [plt.Rectangle((0,0),1,1, facecolor=colors[i], edgecolor='black', alpha=0.8, label=f'Resource {i+1}') for i in range(nb_resource)]
ax1.legend(handles=legend_elements, loc='upper right')

# Show the grid for tasks
ax1.grid(True, axis='x', linestyle='--', alpha=0.7)

# Calculate total resource usage over time
max_time = max(end_times)
resource_usage = np.zeros((nb_resource, max_time + 1))
for i in range(nb_jobs):
    for t in range(start_times[i], end_times[i]):
        for r in range(nb_resource):
            resource_usage[r, t] += resource_needs[i, r]

# Plot total resource usage
for r in range(nb_resource):
    ax2.plot(range(max_time + 1), resource_usage[r], label=f'Resource {r+1}', color=colors[r])

# Plot maximal allowed usage
for r in range(nb_resource):
    ax2.axhline(y=resource_capacities[r], color=colors[r], linestyle='--',
                label=f'Max Resource {r+1}')

ax2.set_xlabel('Time')
ax2.set_ylabel('Total Resource Usage')
ax2.set_title('Total Resource Usage Over Time')
ax2.legend()
ax2.grid(True, linestyle='--', alpha=0.7)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

# Print the makespan (total project duration)
print(f"Makespan: {max(end_times)}")


## Exercise 4
A heterosquare of order n is a n*n square whose elements are distinct integers from
1 to n^2 such that the sums of the rows, columns and diagonals are all different.
Here is an example of heterosquare of order 3
```
           19

1  2  3    6
8  9  4    21
7  6  5    18

16 17 12   15  (Sums)
```
Write a CPMpy Model to solve a heterosquare of order n, and think about the symmetries in its solutions.
hint: without symmetry breaking there are 24960 solutions, with symmetry breaking 3120.

## Exercise 5
Balanced Incomplete Block Design (BIBD)

A BIBD is defined as an arrangement of $v$ distinct objects into b blocks such that each block contains exactly $k$ distinct objects, each object occurs in exactly $r$ different blocks, and every two distinct objects occur together in exactly lambda blocks. Another way of defining a BIBD is in terms of its incidence matrix, which is a $v$ by $b$ binary matrix with exactly $r$ ones per row, $k$ ones per column, and with a scalar product of lambda $l$ between any pair of distinct rows.

Write a CPMpy Model to solve a BIBD and solve the problem with the following data:

In [6]:
# Data
v,b = 7,7
r,k = 3,3
l = 1

Find all solutions of the problem. (don't print all solutions; it has too many)


In [11]:
# Variables, incidence matrix
block = cp.boolvar(shape=(v,b), name="block")

# Constraints on incidence matrix
m = cp.Model(
        [cp.sum(row) == r for row in block],
        [cp.sum(col) == k for col in block.T],
)

# the scalar product of every pair of distinct rows sums up to `l`
for row_a in range(v):
    for row_b in range(row_a+1,v):
        m.add(cp.sum(block[row_a,:] * block[row_b,:]) == l)

def print_solution():
    # pretty print
    print(f"BIBD: {b} obj, {v} blocks, r={r}, k={k}, l={l}")
    for (i, row) in enumerate(block.value()):
        srow = "".join('X ' if e else '  ' for e in row)
        print(f"Object {i + 1}: [ {srow}]")

sols_num = m.solveAll(solution_limit=10000)
print(sols_num)  # we've put a solution limit of 10000

Constraints:
    [sum([block[0,0], block[0,1], block[0,2], block[0,3], block[0,4], block[0,5], block[0,6]]) == 3, sum([block[1,0], block[1,1], block[1,2], block[1,3], block[1,4], block[1,5], block[1,6]]) == 3, sum([block[2,0], block[2,1], block[2,2], block[2,3], block[2,4], block[2,5], block[2,6]]) == 3, sum([block[3,0], block[3,1], block[3,2], block[3,3], block[3,4], block[3,5], block[3,6]]) == 3, sum([block[4,0], block[4,1], block[4,2], block[4,3], block[4,4], block[4,5], block[4,6]]) == 3, sum([block[5,0], block[5,1], block[5,2], block[5,3], block[5,4], block[5,5], block[5,6]]) == 3, sum([block[6,0], block[6,1], block[6,2], block[6,3], block[6,4], block[6,5], block[6,6]]) == 3]
    [sum([block[0,0], block[1,0], block[2,0], block[3,0], block[4,0], block[5,0], block[6,0]]) == 3, sum([block[0,1], block[1,1], block[2,1], block[3,1], block[4,1], block[5,1], block[6,1]]) == 3, sum([block[0,2], block[1,2], block[2,2], block[3,2], block[4,2], block[5,2], block[6,2]]) == 3, sum([block[0,3], 

Print a few solutions and try to detect symmetric solutions. Does this problem have any symmetries? If yes, what type of symmetries? Break the symmetries and notice the reduction in the number of solutions.


There are both row and column symmetries here, we can swap any 2 rows, and any 2 columns.

## Extra's
Find the numbers that can be expressed as the sum of two cubes in two different ways, i.e. the numbers $n=a^3+b^3=c^3+d^3$ for some a, b, c, d between 1 and 1000 $a!=b!=c!=d$.
Again look at all solutions to check symmetric assignments and add to the model the necessary constraints to rule out the symmetric solutions.
Hint: Use Minizinc to solve this model (if you have it installed). Otherwise, you will have to lower the bounds of the variables or wait a (very) long time.

In [ ]:
# optional setup when using Minizinc (this does not work on google colab
! pip install nest_asyncio --quiet
import nest_asyncio
nest_asyncio.apply()